In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyneb as pn

# The extinction class: RedCorr()

The class RedCorr manages the extinction (reddening) correction. It can compute the logarithmic extinction at H$\beta$ by comparing an observed ratio to a theoretical one (usually H$\alpha$/H$\beta$, but any other ratio can be used). The object is also able to compute the correction to be applied to any intensity, given the wavelength of the line.

Various extinction laws are included in the class, and any user-defined function can also be implemented. The available extinction laws can be listed by entering:

In [ ]:
RC = pn.RedCorr()

In [ ]:
RC.printLaws()

In [ ]:
# Print out only the labels for the extinction laws:
RC.getLaws()

## Some definitions

The attenuation of a source $I_{\lambda , 0}$ leads to the observation $I_\lambda$. This attenuation may be writen as: $I_\lambda = I_{\lambda , 0} . e^{-\tau_\lambda}$.

The dust optical depth at $\lambda$ is $\tau_\lambda = K . g(\lambda)$, where $g(\lambda)$ is an universal law (the same for all objects) and $K$ is typical of each object.

The extinction in magnitudes $A_\lambda$ is : $A_\lambda = -2.5 \log_{10}(I_\lambda / I_{\lambda , 0}) = 2.5 \log_{10}(e) . \tau_\lambda = 1.086 \tau_\lambda$. This leads to $I_\lambda = I_{\lambda , 0} 10^{0.4 A_\lambda}$

The relative extinction is $E_{B-V} = A(B) - A(V)$. 

The normalized extinction $1/R_V$ measures the steepness of the extinction: $R_V = A(V) / E_{B-V}$. Its value is classically 3.1.

The extinction can also be written as: $I_\lambda = I_{\lambda , 0} . 10^{-0.4 E_{B-V} X_\lambda} = I_{\lambda , 0} . 10^{(f_\lambda-1).c(H\beta)}$, where $X_\lambda$ and $f(\lambda)$ are universal laws.

The RedCorr object takes care of all this mess ;-)

In [ ]:
# Defining the extinction is easy:
RC.E_BV = 1.34
RC.law = 'F99'

In [ ]:
# All this can be done in a single command:
RC = pn.RedCorr(E_BV = 1.32, R_V = 3.2, law = 'F99')

$c(H\beta)$ and $E_{B-V}$ are related through:
    
$$(1-f_\lambda).c(H\beta) = 0.4 E_{B-V} X_\lambda$$ applied to $\lambda = 4861A$, with $f_\beta$ = 0.
so that, once one of the two parameters is defined, the other is also automatically defined; to output its value, enter:

In [ ]:
RC.cHbeta

This slightly depends on the law you use:

In [ ]:
RC2 = pn.RedCorr(E_BV = 1.32, R_V = 3.2, law = 'CCM89')
RC2.cHbeta

$A_V$ is also available:

In [ ]:
print(RC.AV)

To obtain the correction ot apply at a given wavelength:

In [ ]:
wave = 5007.0
corr = RC.getCorr(wave)
print(corr)

In [ ]:
corr2 = RC2.getCorr(wave)
print(corr2)

The correction may be relative to H$\beta$:

In [ ]:
corr = RC.getCorrHb(wave)
print(corr)

The correction can be defined by setting a theoretical value to be obtained once correction is applied (may be a vector):

In [ ]:
RC.setCorr(obs_over_theo = 3.56 / 2.83, wave1=6563., wave2=4861)
print(RC.AV)
print(RC.cHbeta)
print(RC.E_BV)

Once the RedCor object is defined, the correction can be applied to any number of wavelengths (vectorized).

A plotting tool is available:

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
RC.plot(w_inf=1000.0, w_sup=10000.0, laws = 'all', ax=ax)